In [1]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [2]:
def dbscan_cluster_analysis(data_path, eps_range=(2, 4), min_samples_range=(2, 4)):
    # 讀取Excel文件
    df = pd.read_csv(data_path)
    df = df.dropna(axis=0)

    rename_dict = {
        'A': '政府機構(A)',
        'B': '各級學校(B)',
        'C': '醫療院所(C)',
        'D': '飯店旅館(D)',
        'E': '金融機構(E)',
        'F': '觀光旅遊(F)',
        'G': '休閒娛樂(G)',
        'H': '逛街購物(H)',
        'I': '餐飲小吃(I)',
        'J': '行車服務(J)',
        'K': '交通設施(K)',
        'L': '民間機構(L)',
        'X': '鄰避設施(X)'
    }

    df.rename(columns=rename_dict, inplace=True)

    # 先保存原始資料，以在最後加上總價元和車位總價元
    orginal_data = pd.read_csv(data_path)
    orginal_data = orginal_data.dropna(axis=0)

    df.drop(columns=['總價元'], inplace=True)
    df.drop(columns=['車位總價元'], inplace=True)

    corr_features = set()

    # 建立相關矩陣
    corr_matrix = df.corr()

    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > 0.8:
                colname = corr_matrix.columns[i]
                corr_features.add(colname)

    # Feature Selection
    df.drop(labels=corr_features, axis=1, inplace=True)

    results = []

    for eps in range(eps_range[0], eps_range[1] + 1):
        for min_samples in range(min_samples_range[0], min_samples_range[1] + 1):
            # 資料標準化
            scaler = StandardScaler()
            scaled_features = scaler.fit_transform(df)

            # 擬合 DBSCAN 模型
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            dbscan.fit(scaled_features)

            # 獲取分群結果
            cluster_label = f'分群結果_DBSCAN_eps_{eps}_min_samples_{min_samples}'
            df[cluster_label] = dbscan.labels_
            results.append(cluster_label)

    # 將原始資料加入總價元和車位總價元
    df['總價元'] = orginal_data['總價元']
    df['車位總價元'] = orginal_data['車位總價元']

    return df, results

In [3]:
# 使用範例
result_df, cluster_result_columns = dbscan_cluster_analysis('/Users/apple/Desktop/0405(四版).csv')
print(result_df.head())

   土地移轉總面積平方公尺      移轉層次      總樓層數  建物現況格局-房  建物現況格局-廳  建物現況格局-衛  建物現況格局-隔間  \
0     0.009922 -0.635776 -1.057031  0.412186  0.584395 -0.445601   0.302105   
1    -0.199129  0.330714 -0.319392  0.412186  0.584395 -0.445601   0.302105   
2    -0.208861  1.055582  0.418246 -0.203066 -0.600795 -1.333524   0.302105   
3    -0.015286 -0.877399  0.418246 -0.818318 -0.600795 -0.445601   0.302105   
4    -0.308404  1.780449  0.787065 -0.818318 -0.600795 -0.445601   0.302105   

   有無管理組織  車位移轉總面積平方公尺    附屬建物面積  ...  分群結果_DBSCAN_eps_2_min_samples_3  \
0       0    -0.286596 -0.169071  ...                               -1   
1       1    -0.286596 -0.169071  ...                               -1   
2       1    -0.286596 -0.169071  ...                               -1   
3       1    -0.286596 -0.169071  ...                               -1   
4       1    -0.286596 -0.169071  ...                               -1   

   分群結果_DBSCAN_eps_2_min_samples_4  分群結果_DBSCAN_eps_3_min_samples_2  \
0        